# **Projeto de Previsão de Inadimplência**

---
## **Motivação:**
A inadimplência é um problema crítico para instituições financeiras, uma vez que impacta diretamente o fluxo de caixa e a saúde financeira das empresas. Prever com antecedência quais clientes têm maior chance de se tornarem inadimplentes permite que as empresas adotem estratégias preventivas, minimizando prejuízos.



---
## **Objetivos:**

Este projeto tem como **principal objetivo** construir um modelo de Machine Learning que seja capaz de prever a inadimplência de clientes com base em um conjunto de dados de contratos de financiamento.

- **Construir um modelo preditivo de inadimplência** utilizando algoritmos de classificação.
- **Analisar as variáveis mais relevantes** para prever a inadimplência, ajudando a entender os fatores de risco.
- **Avaliar a performance dos modelos** utilizando métricas de performance.


---

#### Entre as colunas deste dataset temos:
| Coluna                        | Descrição                                          |
|-------------------------------|----------------------------------------------------|
| **NUMERO_CONTRATO**            | Número identificador do contrato                   |
| **DATA_ASSINATURA_CONTRATO**   | Data em que o contrato foi assinado                |
| **TIPO_FINANCIAMENTO**         | Tipo de financiamento oferecido                    |
| **TAXA_AO_ANO**                | Taxa de juros anual do contrato                    |
| **PZ_FINANCIAMENTO**           | Prazo de financiamento (em meses)                  |
| **CIDADE_CLIENTE**             | Cidade onde o cliente reside                       |
| **ESTADO_CLIENTE**             | Estado onde o cliente reside                       |
| **RENDA_MENSAL_CLIENTE**       | Renda mensal do cliente                            |
| **QT_PC_ATRASO**               | Quantidade de parcelas em atraso                   |
| **QT_DIAS_PRIM_PC_ATRASO**     | Quantidade de dias em atraso da primeira parcela   |
| **QT_TOTAL_PC_PAGAS**          | Quantidade total de parcelas pagas                 |
| **VL_TOTAL_PC_PAGAS**          | Valor total das parcelas pagas                     |
| **QT_PC_PAGAS_EM_DIA**         | Quantidade de parcelas pagas em dia                |
| **QT_DIAS_MIN_ATRASO**         | Menor quantidade de dias de atraso                 |
| **QT_DIAS_MAX_ATRASO**         | Maior quantidade de dias de atraso                 |
| **QT_DIAS_MEDIA_ATRASO**       | Média de dias em atraso                            |
| **VALOR_FINANCIAMENTO**        | Valor total do financiamento                       |
| **VALOR_PARCELA**              | Valor de cada parcela                              |
| **IDADE_DATA_ASSINATURA_CONTRATO** | Idade do cliente no momento da assinatura do contrato |
| **INADIMPLENTE_COBRANCA**      | Indicador se o cliente está inadimplente           |



**A coluna alvo é "INADIMPLENTE_COBRANCA"**

#### Este é um problema de **Classificação**

---

## **Etapas do projeto**

1. **Carregamento e análise inicial dos dados**: Vamos carregar o dataset e verificar suas características iniciais.

2. **Exploração e tratamento dos dados**: Identificar dados ausentes, remover outliers (se necessário), e preparar as variáveis para modelagem.

3. **Análise Estatística e Engenharia de Atributos**: Para entender as características e padrões dos dados e melhorar a qualidade dos dados.

4. **Visualização com Gráficos**: Visualizar os dados de uma forma gráfica para ajudar na compreensão.

5. **Treinamento do modelo**: Construir e treinar modelos preditivos usando algoritmos de Machine Learning.

6. **Avaliação do modelo**: Avaliar a performance dos modelos com base nas métricas adequadas.

7. **Colocar o modelo em producão**: Para que as previsões e insights gerados pelo modelo possam ser utilizados em cenários do mundo real.

8. **Conclusão**: Resumo dos resultados e insights finais.

---

### Conectando com o Banco de dados e Importando os dados

In [1722]:
#importando as bibliotecas
import warnings
import pandas as pd
import pypyodbc as sql
import os
from dotenv import load_dotenv

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
warnings.filterwarnings("ignore")

In [1723]:
#carregando as variaveis de ambiente
load_dotenv()

USUARIO = os.environ.get("USUARIO")
SENHA = os.environ.get("SENHA")
HOST = os.environ.get("HOST")
DATABASE = os.environ.get("DATABASE")


In [1724]:
#importar os dados do SQL

connection_string = 'Driver={SQL Server};Server=' + HOST + ';Database=' + DATABASE + ';UID=' + USUARIO + ';PWD=' + SENHA + ';'

conexao = sql.connect(connection_string)

df_original = pd.read_sql_query("SELECT * FROM EXTRACAO_DADOS_SISTEMA", conexao)

conexao.close()

### Começando a Análise Exploratória

In [1725]:
#observando o numero de linhas e colunas
print(f"O dataset contém {df_original.shape[0]} linhas e {df_original.shape[1]} colunas.")

O dataset contém 10415 linhas e 20 colunas.


In [1726]:
#observando todas as colunas e os tipos das mesmas, depois, iremos otimizar este dataframe
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   numero_contrato                 10415 non-null  int64  
 1   data_assinatura_contrato        10415 non-null  object 
 2   tipo_financiamento              10415 non-null  object 
 3   taxa_ao_ano                     10415 non-null  float64
 4   pz_financiamento                10415 non-null  int64  
 5   cidade_cliente                  10415 non-null  object 
 6   estado_cliente                  10415 non-null  object 
 7   renda_mensal_cliente            10415 non-null  float64
 8   qt_pc_atraso                    10415 non-null  int64  
 9   qt_dias_prim_pc_atraso          10415 non-null  int64  
 10  qt_total_pc_pagas               10415 non-null  int64  
 11  vl_total_pc_pagas               10411 non-null  float64
 12  qt_pc_paga_em_dia               

In [1727]:
#visao geral dos dados
df_original.sample(10)

,numero_contrato,data_assinatura_contrato,tipo_financiamento,taxa_ao_ano,pz_financiamento,cidade_cliente,estado_cliente,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato,inadimplente_cobranca
6073,70977,2013-02-08,IMOBILIARIO,17.00,240,BRASILIA,DF,7800.00,0,0,118,248446.29,80,1,163,16,600000.00,2925.00,29.00,SIM
3297,50953,2012-09-14,IMOBILIARIO,16.00,200,BRASILIA,DF,5800.00,0,0,107,188039.24,92,2,43,11,300000.00,1740.00,59.00,NAO
7953,80055,2017-02-21,IMOBILIARIO,17.00,72,SAO PAULO,SP,9800.00,0,0,22,16273.47,22,0,0,0,280000.00,3983.33,34.00,SIM
8857,75983,2015-03-24,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,37,2600,5,3097.61,5,0,0,0,266868.00,3796.52,40.00,NAO
9932,76567,2012-02-10,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,68,3797,4,1611.13,4,0,0,0,175000.00,2486.11,29.00,SIM
503,50363,2018-05-11,IMOBILIARIO,16.00,180,JATAI,GO,5800.00,0,0,55,88564.00,41,1,11,4,170294.00,1097.45,39.00,SIM
6635,79097,2016-03-11,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,51,1942,16,9080.68,12,1,31,11,245000.00,3485.42,26.00,NAO
2217,79230,2016-04-12,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,58,2153,8,7678.31,7,4,4,4,315000.00,4481.25,48.00,SIM
7965,153692,2022-07-18,IMOBILIARIO,19.05,48,CONTAGEM,MG,1800.00,0,0,3,1637.04,2,0,0,0,192500.00,4119.56,22.00,SIM
3288,77534,2014-06-13,IMOBILIARIO,15.00,72,BRASILIA,DF,7800.00,44,2215,28,10296.50,12,1,70,28,280000.00,3972.22,48.00,NAO


In [1728]:
#vendo metricas das colunas numericas
df_original.describe()

,numero_contrato,taxa_ao_ano,pz_financiamento,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato
count,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10411.00,10415.00,10415.00,10415.00,10415.00,10415.00,10415.00,10414.00
mean,95569.36,17.27,114.24,5620.26,16.72,664.39,35.95,44528.55,27.67,6.12,79.13,28.84,325590.86,3617.34,40.79
std,34561.66,2.57,64.33,2930.30,30.71,1181.35,28.38,65640.97,25.15,82.91,334.87,137.40,177477.49,1932.71,12.39
min,32709.00,7.00,48.00,1800.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,29327.00,185.74,0.00
25%,75868.50,16.00,72.00,1800.00,0.00,0.00,11.00,8675.44,7.00,0.00,0.00,0.00,210000.00,2467.76,31.25
50%,79111.00,17.00,72.00,7800.00,0.00,0.00,33.00,24345.60,22.00,1.00,8.00,4.00,280000.00,3470.59,39.00
75%,136637.50,19.00,180.00,7800.00,22.00,1005.50,54.00,46787.29,42.00,3.00,62.00,22.00,392973.77,4930.25,49.00
max,155890.00,25.50,240.00,9800.00,178.00,5655.00,167.00,714499.74,136.00,5465.00,5710.00,5480.00,1400000.00,14410.00,118.00


- Aqui observamos alguns potenciais outliers como idade_data_assinatura_contrato minima: 0 e idade_data_assinatura_contrato maxima: 118
---

In [1729]:
#vendo metricas das colunas nao numericas
df_original.describe(exclude="number")

,data_assinatura_contrato,tipo_financiamento,cidade_cliente,estado_cliente,inadimplente_cobranca
count,10415,10415,10415,10415,10415
unique,1070,1,525,27,2
top,2014-06-13,IMOBILIARIO,BRASILIA,DF,SIM
freq,187,10415,3737,4240,8038


In [1730]:
#filtro para ver a quantidade media de parcelas em atraso por renda mensal do cliente
filtro_atraso = df_original.groupby("renda_mensal_cliente")["qt_pc_atraso"].mean()
print(filtro_atraso)

maior_id_atraso = filtro_atraso.idxmax()
maior_media_atraso = filtro_atraso.max()

print(f"\nA renda_mensal_cliente com a maior média de qt_pc_atraso é R$ {maior_id_atraso} com uma média de {maior_media_atraso:.2f} de parcelas em atraso")

renda_mensal_cliente
1800.00    5.60
3800.00    0.06
4800.00    8.12
5800.00   31.69
6800.00   26.58
7800.00   22.82
9800.00   16.73
Name: qt_pc_atraso, dtype: float64

A renda_mensal_cliente com a maior média de qt_pc_atraso é R$ 5800.0 com uma média de 31.69 de parcelas em atraso


- A quantidade média de parcelas atrasadas é bem maior quando o cliente tem uma renda mensal de mais de **R$ 5.800.**

---

In [1731]:
#filtro para ver a idade media do cliente na data de assinatura do contrato por renda mensal do cliente
df_original.groupby("renda_mensal_cliente")["idade_data_assinatura_contrato"].mean()

renda_mensal_cliente
1800.00   40.98
3800.00   45.00
4800.00   42.25
5800.00   41.90
6800.00   44.77
7800.00   40.37
9800.00   39.48
Name: idade_data_assinatura_contrato, dtype: float64

- A média de idade na hora da assinatura do contrato é bem parecida em todos os níveis de renda com a média total.

---

In [1732]:
#filtro para ver a quantidade media do valor das parcelas por renda mensal do cliente
filtro_renda = df_original.groupby("renda_mensal_cliente")["valor_parcela"].mean()
print(filtro_renda)

maior_id_venda = filtro_renda.idxmax()
maior_media_venda = filtro_renda.max()

print(f"\nA renda_mensal_cliente com a maior média de valor_parcela é R$ {maior_id_venda} com uma média de R$ {maior_media_venda:.2f} por parcela")



renda_mensal_cliente
1800.00   4733.81
3800.00   1143.67
4800.00   1040.42
5800.00   1265.53
6800.00   2843.32
7800.00   3498.73
9800.00   3910.99
Name: valor_parcela, dtype: float64

A renda_mensal_cliente com a maior média de valor_parcela é R$ 1800.0 com uma média de R$ 4733.81 por parcela


- A média do valor da parcela por renda mensal do cliente é bem maior quando o cliente recebe **R$ 1.800 ou menos**, ou, quando o cliente recebe **R$ 6.800 ou mais**.

---

In [1733]:
#observando o inicio e fim dos dados da coluna data_assinatura_contrato
inicio = pd.to_datetime(df_original["data_assinatura_contrato"]).dt.date.min() 
fim = pd.to_datetime(df_original["data_assinatura_contrato"]).dt.date.max()

print("Período dos dados de:", inicio, "até", fim)

Período dos dados de: 2012-01-10 até 2022-11-14


- Aqui observamos que só existe um tipo de financiamento nestes dados: IMOBILIARIO
- Clientes de 525 cidades diferentes
- Clientes de todos os estados do Brasil
- Maior parte dos clientes neste conjunto de dados está inadimplente (8038 de 10415, ~77%)
- Período dos dados de: 2012-01-10 até 2022-11-14


---

In [1734]:
#filtro para ver a porcentagem de valores por renda mensal do cliente
print(df_original["renda_mensal_cliente"].value_counts(normalize=True))
print(f"\nA maior parte dos clientes ({df_original["renda_mensal_cliente"].value_counts().max()}), recebe entre R$ 6.900 e R$ {df_original["renda_mensal_cliente"].value_counts().idxmax()}")

renda_mensal_cliente
7800.00   0.41
1800.00   0.34
5800.00   0.10
9800.00   0.09
4800.00   0.04
6800.00   0.01
3800.00   0.00
Name: proportion, dtype: float64

A maior parte dos clientes (4306), recebe entre R$ 6.900 e R$ 7800.0


- A maior parte dos clientes (**41%**) recebe entre **R$ 6.900 e R$ 7.800 mensais**
- **34%** dos clientes recebem até **R$ 1.800**
- **75%** dos cliente ou recebem entre **R$ 6.900 e R$ 7.800 ou até R$ 1.800**
          
---

In [1735]:
#visualizando a quantidade de linhas onde a idade na data de assinatura é maior ou menor que um valor
total = len(df_original.query("idade_data_assinatura_contrato < 10"))
total2 = len(df_original.query("idade_data_assinatura_contrato < 20"))
total3 = len(df_original.query("idade_data_assinatura_contrato > 50"))
total4 = len(df_original.query("idade_data_assinatura_contrato > 70"))
print(f"Número de linhas onde a idade_data_assinatura_contrato é menor que 10: {total}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é menor que 20: {total2}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é maior que 50: {total3}")
print(f"Número de linhas onde a idade_data_assinatura_contrato é maior que 70: {total4}")


Número de linhas onde a idade_data_assinatura_contrato é menor que 10: 7
Número de linhas onde a idade_data_assinatura_contrato é menor que 20: 80
Número de linhas onde a idade_data_assinatura_contrato é maior que 50: 2303
Número de linhas onde a idade_data_assinatura_contrato é maior que 70: 175


- Existem **7** registros onde a idade na data de assinatura é menor que 10 anos, um possível erro 
- **80** clientes com menos de **20 anos** na data de assinatura
- **2303** clientes com mais de **50 anos** na data de assinatura
- **175** clientes com mais de **70 anos** na data de assinatura
          
---

In [1736]:
#porcentagem de parcelas pagas em dia e em atraso

total_pagas = df_original["qt_total_pc_pagas"].sum()
pagas_em_dia = df_original["qt_pc_paga_em_dia"].sum()

porcentagem_em_dia = (pagas_em_dia / total_pagas) * 100
porcentagem_atraso = 100 - porcentagem_em_dia

print(f"Porcentagem de parcelas pagas em dia: {porcentagem_em_dia:.2f}%")
print(f"Porcentagem de parcelas pagas em atraso: {porcentagem_atraso:.2f}%")


Porcentagem de parcelas pagas em dia: 76.97%
Porcentagem de parcelas pagas em atraso: 23.03%


- **76%** das parcelas pagas, são pagas em dia
---

In [1737]:
#verificando a quantidade de valores únicos de cada coluna
df_original.nunique()

numero_contrato                   10415
data_assinatura_contrato           1070
tipo_financiamento                    1
taxa_ao_ano                         111
pz_financiamento                      9
cidade_cliente                      525
estado_cliente                       27
renda_mensal_cliente                  7
qt_pc_atraso                        130
qt_dias_prim_pc_atraso              227
qt_total_pc_pagas                   137
vl_total_pc_pagas                  9860
qt_pc_paga_em_dia                   129
qt_dias_min_atraso                  122
qt_dias_max_atraso                  534
qt_dias_media_atraso                323
valor_financiamento                 834
valor_parcela                      1826
idade_data_assinatura_contrato       78
inadimplente_cobranca                 2
dtype: int64

- Verificamos novamente que **tipo_financiamento** possuí apenas um tipo de dado
- **vl_total_pc_pagas** possuí alta cardinalidade 
- Apenas 7 tipos diferentes de **renda_mensal_cliente**                
---

Faremos mais análises na parte de **Visualização com gráficos**


### Otimização do dataframe

In [1738]:
#otimizando a coluna Data_Contratacao para datetime64[ns]                  
df_original["data_assinatura_contrato"] = pd.to_datetime(df_original["data_assinatura_contrato"])

In [1739]:
#otimizando colunas categoricas
colunas_categoricas = ["tipo_financiamento", "cidade_cliente", "estado_cliente", "inadimplente_cobranca"]

for coluna in colunas_categoricas:
	df_original[coluna] = df_original[coluna].astype("category")


In [1740]:
#otimizando colunas numericas
colunas_float = df_original.select_dtypes(include="float64").columns
colunas_int = df_original.select_dtypes(include="int64").columns

df_original[colunas_float] = df_original[colunas_float].apply(pd.to_numeric, downcast="float")
df_original[colunas_int] = df_original[colunas_int].apply(pd.to_numeric, downcast="integer")


In [1741]:
#verificando a mudança de tamanho do dataset
df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   numero_contrato                 10415 non-null  int32         
 1   data_assinatura_contrato        10415 non-null  datetime64[ns]
 2   tipo_financiamento              10415 non-null  category      
 3   taxa_ao_ano                     10415 non-null  float32       
 4   pz_financiamento                10415 non-null  int16         
 5   cidade_cliente                  10415 non-null  category      
 6   estado_cliente                  10415 non-null  category      
 7   renda_mensal_cliente            10415 non-null  float32       
 8   qt_pc_atraso                    10415 non-null  int16         
 9   qt_dias_prim_pc_atraso          10415 non-null  int16         
 10  qt_total_pc_pagas               10415 non-null  int16         
 11  vl

- O tamanho do dataframe diminuiu de 1.6+ MB para 683.0 KB
---

### Tratamento dos dados

In [1742]:
#removendo linhas duplicadas 
df_original.drop_duplicates()
print(f"O dataset contém {df_original.shape[0]} linhas e {df_original.shape[1]} colunas.")

O dataset contém 10415 linhas e 20 colunas.


- Não foram encontradas linhas duplicadas
---

In [1743]:
#verificando valores nulos
print(f"O número de valores vazios de cada coluna \n\n{df_original.isnull().sum()}")

O número de valores vazios de cada coluna 

numero_contrato                   0
data_assinatura_contrato          0
tipo_financiamento                0
taxa_ao_ano                       0
pz_financiamento                  0
cidade_cliente                    0
estado_cliente                    0
renda_mensal_cliente              0
qt_pc_atraso                      0
qt_dias_prim_pc_atraso            0
qt_total_pc_pagas                 0
vl_total_pc_pagas                 4
qt_pc_paga_em_dia                 0
qt_dias_min_atraso                0
qt_dias_max_atraso                0
qt_dias_media_atraso              0
valor_financiamento               0
valor_parcela                     0
idade_data_assinatura_contrato    1
inadimplente_cobranca             0
dtype: int64


- Identificamos que ***vl_total_pc_pagas**  e **idade_data_assinatura_contrato** possuem 4 e 1 valor vazio respectivamente, iremos excluir essas linhas já que são poucas e não irão afetar o modelo
---

In [1744]:
#excluindo linhas com valores nulos
df_original.dropna(inplace=True)

In [1745]:
#verificando novamente se existem valores nulos
print(f"O número de valores vazios de cada coluna \n\n{df_original.isnull().sum()}")


O número de valores vazios de cada coluna 

numero_contrato                   0
data_assinatura_contrato          0
tipo_financiamento                0
taxa_ao_ano                       0
pz_financiamento                  0
cidade_cliente                    0
estado_cliente                    0
renda_mensal_cliente              0
qt_pc_atraso                      0
qt_dias_prim_pc_atraso            0
qt_total_pc_pagas                 0
vl_total_pc_pagas                 0
qt_pc_paga_em_dia                 0
qt_dias_min_atraso                0
qt_dias_max_atraso                0
qt_dias_media_atraso              0
valor_financiamento               0
valor_parcela                     0
idade_data_assinatura_contrato    0
inadimplente_cobranca             0
dtype: int64


- Todos os valores nulos foram excluídos
---

In [1746]:
#verificando a possibilidade de outliers
df_original[df_original["idade_data_assinatura_contrato"] <= 17]

,numero_contrato,data_assinatura_contrato,tipo_financiamento,taxa_ao_ano,pz_financiamento,cidade_cliente,estado_cliente,renda_mensal_cliente,qt_pc_atraso,qt_dias_prim_pc_atraso,qt_total_pc_pagas,vl_total_pc_pagas,qt_pc_paga_em_dia,qt_dias_min_atraso,qt_dias_max_atraso,qt_dias_media_atraso,valor_financiamento,valor_parcela,idade_data_assinatura_contrato,inadimplente_cobranca
856,79171,2017-11-14,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,35,35294.51,10,1,55,30,412755.00,5871.93,9.00,SIM
1418,79545,2016-11-11,IMOBILIARIO,20.00,72,BRASILIA,DF,7800.00,0,0,38,33545.05,3,1,245,61,420000.00,6000.00,8.00,SIM
2044,137914,2019-04-09,IMOBILIARIO,14.65,72,PLANALTINA DF,DF,1800.00,0,0,44,31670.84,42,2,2,2,385000.00,5459.13,12.00,SIM
2219,79450,2016-04-12,IMOBILIARIO,17.00,72,BRASILIA,DF,7800.00,0,0,53,22320.79,41,1,82,21,210000.00,2987.50,17.00,SIM
2253,76537,2012-02-10,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,0,0,56,28877.66,44,11,178,61,175000.00,2486.11,16.00,SIM
2979,76481,2012-02-10,IMOBILIARIO,16.00,72,BRASILIA,DF,7800.00,0,0,63,35871.16,44,8,68,34,175000.00,2486.11,16.00,SIM
3002,136455,2018-12-11,IMOBILIARIO,14.65,72,SAO PAULO,SP,1800.00,0,0,43,27952.17,37,2,21,5,280000.00,3970.28,0.00,SIM
3142,75101,2012-01-10,IMOBILIARIO,15.00,72,PLANALTINA,GO,7800.00,59,3769,6,5106.58,4,20,49,34,373590.00,5299.94,17.00,NAO
3562,136456,2018-12-11,IMOBILIARIO,14.65,72,SAO PAULO,SP,1800.00,0,0,43,27955.99,37,2,21,8,280000.00,3970.28,0.00,SIM
3749,78832,2016-02-16,IMOBILIARIO,17.00,72,ALTAMIRA,PA,7800.00,0,0,51,34921.82,11,1,154,43,210000.00,2987.50,7.00,SIM


- Alguns destes valores aparentam ser outliers, mas como são poucos, eles serão tratados na parte de Normalização/ Padronização dos dados
---

In [1747]:
#eliminando colunas inúteis para a nossa análise

df_original = df_original.drop(["numero_contrato", "data_assinatura_contrato", "tipo_financiamento"], axis=1)


- Eliminando **numero_contrato** pois não tem relevância para nossa análise
- Eliminando **data_assinatura_contrato** pois não se trata de uma série temporal
- Eliminando **tipo_financiamento** pois todas as linhas possuem o mesmo valor

---

In [1748]:
#transformando nossas colunas para caixa alta
df_original.columns = df_original.columns.str.upper()

In [1749]:
#renomeando algumas colunas

df_original.rename(columns={"VALOR_FINANCIAMENTO": "VL_FINANCIAMENTO", "VALOR_PARCELA": "VL_PARCELA",  }, inplace=True)

In [1750]:
#visualizando novamente as colunas

df_original.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   TAXA_AO_ANO                     10410 non-null  float32 
 1   PZ_FINANCIAMENTO                10410 non-null  int16   
 2   CIDADE_CLIENTE                  10410 non-null  category
 3   ESTADO_CLIENTE                  10410 non-null  category
 4   RENDA_MENSAL_CLIENTE            10410 non-null  float32 
 5   QT_PC_ATRASO                    10410 non-null  int16   
 6   QT_DIAS_PRIM_PC_ATRASO          10410 non-null  int16   
 7   QT_TOTAL_PC_PAGAS               10410 non-null  int16   
 8   VL_TOTAL_PC_PAGAS               10410 non-null  float64 
 9   QT_PC_PAGA_EM_DIA               10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO              10410 non-null  int16   
 11  QT_DIAS_MAX_ATRASO              10410 non-null  int16   
 12  QT_DIAS_MEDIA_ATRASO   

## Engenharia de Atributos

In [1751]:
#verificando a cardinalidade das colunas
df_original.nunique()

TAXA_AO_ANO                        111
PZ_FINANCIAMENTO                     9
CIDADE_CLIENTE                     525
ESTADO_CLIENTE                      27
RENDA_MENSAL_CLIENTE                 7
QT_PC_ATRASO                       130
QT_DIAS_PRIM_PC_ATRASO             227
QT_TOTAL_PC_PAGAS                  136
VL_TOTAL_PC_PAGAS                 9859
QT_PC_PAGA_EM_DIA                  129
QT_DIAS_MIN_ATRASO                 122
QT_DIAS_MAX_ATRASO                 534
QT_DIAS_MEDIA_ATRASO               323
VL_FINANCIAMENTO                   834
VL_PARCELA                        1826
IDADE_DATA_ASSINATURA_CONTRATO      78
INADIMPLENTE_COBRANCA                2
dtype: int64

In [1752]:
#verificando valores minimos e maximos para algumas colunas 

df_original[["VL_TOTAL_PC_PAGAS", "VL_FINANCIAMENTO", "VL_PARCELA"]].describe().loc[["min", "mean", "max", "25%", "50%", "75%"]]


,VL_TOTAL_PC_PAGAS,VL_FINANCIAMENTO,VL_PARCELA
min,0.01,29327.00,185.74
mean,44530.61,325570.97,3617.72
max,714499.74,1400000.00,14410.00
25%,8674.22,210000.00,2471.04
50%,24345.61,280000.00,3470.59
75%,46807.04,392973.77,4931.86


In [1753]:
#criando faixa de valores

faixas = [-100, 8_675, 25_000, 47_000, 9_999_999]
categorias = ["Até R$ 8.675", "de R$ 8.676 até R$ 25.000", "de R$ 25.001 até R$ 47.000", "Mais de R$ 47.000"]
df_original["FAIXA_VL_TOTAL_PC_PAGAS"] = pd.cut(df_original["VL_TOTAL_PC_PAGAS"], bins=faixas, labels=categorias)
df_original["FAIXA_VL_TOTAL_PC_PAGAS"] = df_original["FAIXA_VL_TOTAL_PC_PAGAS"].cat.set_categories(categorias, ordered = True)
print(f"É ordenada? : {df_original["FAIXA_VL_TOTAL_PC_PAGAS"].cat.ordered}")
print(df_original["FAIXA_VL_TOTAL_PC_PAGAS"].unique())


faixas = [-100, 210_000, 290_000, 400_000, 9_999_999]
categorias = ["Até R$ 210.000", "de R$ 210.000 até R$ 290.000", "de R$ 290.001 até R$ 400.000", "Mais de R$ 400.000"]
df_original["FAIXA_VALOR_FINANCIAMENTO"] = pd.cut(df_original["VL_FINANCIAMENTO"], bins=faixas, labels=categorias)
df_original["FAIXA_VALOR_FINANCIAMENTO"] = df_original["FAIXA_VALOR_FINANCIAMENTO"].cat.set_categories(categorias, ordered = True)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_VALOR_FINANCIAMENTO"].cat.ordered}")
print(df_original["FAIXA_VALOR_FINANCIAMENTO"].unique())

faixas = [-100, 2_500, 3_500, 5_000, 999999]
categorias = ["Até R$ 2.500", "de R$ 2.501 até R$ 3.500", "de R$ 3.501 até R$ 5.000", "Mais de R$ 5.000"]
df_original["FAIXA_VALOR_PARCELA"] = pd.cut(df_original["VL_PARCELA"], bins=faixas, labels=categorias)
print("\n")
print(f"É ordenada? : {df_original["FAIXA_VALOR_PARCELA"].cat.ordered}")
print(df_original["FAIXA_VALOR_PARCELA"].unique())

É ordenada? : True
['Mais de R$ 47.000', 'de R$ 25.001 até R$ 47.000', 'de R$ 8.676 até R$ 25.000', 'Até R$ 8.675']
Categories (4, object): ['Até R$ 8.675' < 'de R$ 8.676 até R$ 25.000' < 'de R$ 25.001 até R$ 47.000' < 'Mais de R$ 47.000']


É ordenada? : True
['Até R$ 210.000', 'de R$ 290.001 até R$ 400.000', 'Mais de R$ 400.000', 'de R$ 210.000 até R$ 290.000']
Categories (4, object): ['Até R$ 210.000' < 'de R$ 210.000 até R$ 290.000' < 'de R$ 290.001 até R$ 400.000' < 'Mais de R$ 400.000']


É ordenada? : True
['Até R$ 2.500', 'de R$ 2.501 até R$ 3.500', 'de R$ 3.501 até R$ 5.000', 'Mais de R$ 5.000']
Categories (4, object): ['Até R$ 2.500' < 'de R$ 2.501 até R$ 3.500' < 'de R$ 3.501 até R$ 5.000' < 'Mais de R$ 5.000']


- Criamos faixas de valores em algumas colunas e as ordenamos para diminuirmos a cardinalidade delas
---

In [1754]:
#verificando os intervalos
print(df_original.groupby(["FAIXA_VL_TOTAL_PC_PAGAS"]).size())
print("\n")
print(df_original.groupby(["FAIXA_VALOR_FINANCIAMENTO"]).size())
print("\n")
print(df_original.groupby(["FAIXA_VALOR_PARCELA"]).size())

FAIXA_VL_TOTAL_PC_PAGAS
Até R$ 8.675                  2603
de R$ 8.676 até R$ 25.000     2702
de R$ 25.001 até R$ 47.000    2512
Mais de R$ 47.000             2593
dtype: int64


FAIXA_VALOR_FINANCIAMENTO
Até R$ 210.000                  3411
de R$ 210.000 até R$ 290.000    2350
de R$ 290.001 até R$ 400.000    2276
Mais de R$ 400.000              2373
dtype: int64


FAIXA_VALOR_PARCELA
Até R$ 2.500                3233
de R$ 2.501 até R$ 3.500    2722
de R$ 3.501 até R$ 5.000    2603
Mais de R$ 5.000            1852
dtype: int64


In [1755]:
#excluindo as colunas antigas
colunas = ["CIDADE_CLIENTE", "ESTADO_CLIENTE", "RENDA_MENSAL_CLIENTE", "IDADE_DATA_ASSINATURA_CONTRATO", "TAXA_AO_ANO", "PZ_FINANCIAMENTO", "QT_PC_ATRASO", 
           "QT_DIAS_PRIM_PC_ATRASO", "QT_TOTAL_PC_PAGAS", "QT_PC_PAGA_EM_DIA", "QT_DIAS_MIN_ATRASO", "QT_DIAS_MEDIA_ATRASO", "QT_DIAS_MAX_ATRASO", 
           "FAIXA_VL_TOTAL_PC_PAGAS", "FAIXA_VALOR_FINANCIAMENTO", "FAIXA_VALOR_PARCELA", "INADIMPLENTE_COBRANCA"]

df_tratado = pd.DataFrame(df_original, columns = colunas)

In [1756]:
#olhando o dataset para verificarmos a exclusao das colunas antigas e criacao das novas colunas
df_tratado.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   CIDADE_CLIENTE                  10410 non-null  category
 1   ESTADO_CLIENTE                  10410 non-null  category
 2   RENDA_MENSAL_CLIENTE            10410 non-null  float32 
 3   IDADE_DATA_ASSINATURA_CONTRATO  10410 non-null  float32 
 4   TAXA_AO_ANO                     10410 non-null  float32 
 5   PZ_FINANCIAMENTO                10410 non-null  int16   
 6   QT_PC_ATRASO                    10410 non-null  int16   
 7   QT_DIAS_PRIM_PC_ATRASO          10410 non-null  int16   
 8   QT_TOTAL_PC_PAGAS               10410 non-null  int16   
 9   QT_PC_PAGA_EM_DIA               10410 non-null  int16   
 10  QT_DIAS_MIN_ATRASO              10410 non-null  int16   
 11  QT_DIAS_MEDIA_ATRASO            10410 non-null  int16   
 12  QT_DIAS_MAX_ATRASO     

In [1757]:
#visualizando os dados
df_tratado.sample(10)

,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,IDADE_DATA_ASSINATURA_CONTRATO,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
680,SAO PAULO,SP,9800.00,28.00,19.00,72,0,0,38,35,3,3,3,de R$ 8.676 até R$ 25.000,Até R$ 210.000,de R$ 2.501 até R$ 3.500,SIM
2457,SAO PAULO,SP,1800.00,23.00,19.50,72,0,0,26,22,1,1,3,de R$ 8.676 até R$ 25.000,de R$ 210.000 até R$ 290.000,de R$ 2.501 até R$ 3.500,NAO
1917,UBERLANDIA,MG,7800.00,27.00,17.00,72,60,2215,7,4,4,4,5,Até R$ 8.675,Mais de R$ 400.000,Mais de R$ 5.000,NAO
305,SAO PAULO,SP,9800.00,48.00,20.00,72,0,0,50,11,6,69,113,de R$ 25.001 até R$ 47.000,de R$ 210.000 até R$ 290.000,de R$ 3.501 até R$ 5.000,SIM
6824,SAO PAULO,SP,9800.00,29.00,17.00,72,0,0,51,50,6,6,6,de R$ 25.001 até R$ 47.000,de R$ 210.000 até R$ 290.000,de R$ 3.501 até R$ 5.000,SIM
5121,BRASILIA,DF,7800.00,36.00,18.00,240,0,0,88,82,2,20,87,Mais de R$ 47.000,Mais de R$ 400.000,de R$ 2.501 até R$ 3.500,SIM
5136,SAO PAULO,SP,1800.00,74.00,20.00,100,0,0,62,56,1,2,6,Mais de R$ 47.000,Mais de R$ 400.000,Mais de R$ 5.000,SIM
6488,SANTA RITA,PB,7800.00,22.00,20.00,72,0,0,24,12,1,13,25,de R$ 8.676 até R$ 25.000,Até R$ 210.000,de R$ 2.501 até R$ 3.500,SIM
5388,RIO DE JANEIRO,RJ,7800.00,69.00,17.00,72,0,0,69,65,4,9,24,Mais de R$ 47.000,de R$ 290.001 até R$ 400.000,de R$ 3.501 até R$ 5.000,NAO
10411,BRASILIA,DF,5800.00,62.00,18.00,200,122,3703,4,4,0,0,0,Até R$ 8.675,Até R$ 210.000,Até R$ 2.500,SIM


## Análise Estatística

In [1758]:
#verificando novamente as métricas para cada coluna
df_tratado.describe() 

,RENDA_MENSAL_CLIENTE,IDADE_DATA_ASSINATURA_CONTRATO,TAXA_AO_ANO,PZ_FINANCIAMENTO,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MEDIA_ATRASO,QT_DIAS_MAX_ATRASO
count,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00,10410.00
mean,5619.98,40.79,17.27,114.21,16.69,663.71,35.96,27.68,6.12,28.85,79.16
std,2930.53,12.39,2.57,64.31,30.68,1180.94,28.38,25.15,82.93,137.43,334.95
min,1800.00,0.00,7.00,48.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,1800.00,31.25,16.00,72.00,0.00,0.00,11.00,7.00,0.00,0.00,0.00
50%,7800.00,39.00,17.00,72.00,0.00,0.00,33.00,22.00,1.00,4.00,8.00
75%,7800.00,49.00,19.00,180.00,22.00,990.00,54.00,42.00,3.00,22.00,62.00
max,9800.00,118.00,25.50,240.00,178.00,5655.00,167.00,136.00,5465.00,5480.00,5710.00


In [1759]:
df_tratado.describe(exclude="number")

,CIDADE_CLIENTE,ESTADO_CLIENTE,FAIXA_VL_TOTAL_PC_PAGAS,FAIXA_VALOR_FINANCIAMENTO,FAIXA_VALOR_PARCELA,INADIMPLENTE_COBRANCA
count,10410,10410,10410,10410,10410,10410
unique,525,27,4,4,4,2
top,BRASILIA,DF,de R$ 8.676 até R$ 25.000,Até R$ 210.000,Até R$ 2.500,SIM
freq,3734,4237,2702,3411,3233,8035


In [1760]:
#verificando a distribuicao de valores da coluna PZ_Financiamento
print(df_original.groupby(["PZ_FINANCIAMENTO"]).size())

PZ_FINANCIAMENTO
48      343
72     6121
100     938
160       2
180     765
194       2
200     558
220     773
240     908
dtype: int64


- A mediana da **Taxa ao Ano** é de **17.27%**.
- Média de **Prazo de Financiamento** é de **114** enquanto a mediana é de **72**. O pulo de 25% dos dados para 50% dos dados não existe, mas de 50% para 75% sai de **72** para **180**. O desvio padrão também é alto, o que indica uma maior dispersão dos dados.
- Algo semelhante occore com as demais colunas do Dataset, onde a maior parte dos valores está concentrada em uma pequena faixa de valores.
- Maior parte dos clientes é de **Brasília**.
- A faixa do **Valor Total de Parcelas Pagas** que mais aparece é de **R$ 8.676 até R$ 25.000**.
- A faixa do **Valor de Financiamento** que mais aparece é de **Até R$ 210.000**.
- A faixa do **Valor da Parcela** que mais aparece é de **Até R$ 2.500**.	
---

## Visualização com Gráficos

## Conclusão
O modelo de previsão de inadimplência alcançou uma precisão de X%, com uma boa separação entre clientes inadimplentes e não inadimplentes. 

As variáveis mais importantes foram:
- **A**
- **A**
